In [10]:
# Dependencies
import os
import pandas as pd
import config
from sqlalchemy import create_engine

# Extract 

### California, Oregon, Washington Weather
From website https://www.wunderground.com, copied html table element and converted to csv using http://convertcsv.com/html-table-to-csv.htm

In [2]:
#Data sources
#read csv's into pandas
df17ca=pd.read_csv('weather2017.csv',names = ["Max", "Avg", "Min",'Month'])
df16ca=pd.read_csv('weather2016.csv',names = ["Max", "Avg", "Min",'Month'])
df15ca=pd.read_csv('weather2015.csv',names = ["Max", "Avg", "Min",'Month'])

df17or=pd.read_csv('or_weather2017.csv',names = ["Max", "Avg", "Min",'Month'])
df16or=pd.read_csv('or_weather2016.csv',names = ["Max", "Avg", "Min",'Month'])
df15or=pd.read_csv('or_weather2015.csv',names = ["Max", "Avg", "Min",'Month'])

df17wa=pd.read_csv('wa_weather2017.csv',names = ["Max", "Avg", "Min",'Month'])
df16wa=pd.read_csv('wa_weather2016.csv',names = ["Max", "Avg", "Min",'Month'])
df15wa=pd.read_csv('wa_weather2015.csv',names = ["Max", "Avg", "Min",'Month'])

# Transform, Clean data
Create new tables with only the columns needed, groupby and average to get one value for Min, Max and Avg for every year (2015-2017) and every state (CA,OR,WA).

In [3]:
#reset index
df17ca = df17ca.reset_index()
df16ca = df16ca.reset_index()
df15ca = df15ca.reset_index()

df17or = df17or.reset_index()
df16or = df16or.reset_index()
df15or = df15or.reset_index()

df17wa = df17wa.reset_index()
df16wa = df16wa.reset_index()
df15wa = df15wa.reset_index()

In [4]:
#create new table by pulling out only the columns that we need
new_df17ca = df17ca[['index','Avg']].copy()
new_df16ca = df16ca[['index','Avg']].copy()
new_df15ca = df15ca[['index','Avg']].copy()

new_df17or = df17or[['index','Avg']].copy()
new_df16or = df16or[['index','Avg']].copy()
new_df15or = df15or[['index','Avg']].copy()

new_df17wa = df17wa[['index','Avg']].copy()
new_df16wa = df16wa[['index','Avg']].copy()
new_df15wa = df15wa[['index','Avg']].copy()

In [5]:
#group by avg, max and min temp and find one average for each year

cadf17=new_df17ca.groupby('index').mean().reset_index()
cadf16=new_df16ca.groupby('index').mean().reset_index()
cadf15=new_df15ca.groupby('index').mean().reset_index()

ordf17=new_df17or.groupby('index').mean().reset_index()
ordf16=new_df16or.groupby('index').mean().reset_index()
ordf15=new_df15or.groupby('index').mean().reset_index()

wadf17=new_df17wa.groupby('index').mean().reset_index()
wadf16=new_df16wa.groupby('index').mean().reset_index()
wadf15=new_df15wa.groupby('index').mean().reset_index()

cadf17['yr']='2017'
cadf17['State']='California'
cadf16['yr']='2016'
cadf16['State']='California'
cadf15['yr']='2015'
cadf15['State']='California'

ordf17['yr']='2017'
ordf17['State']='Oregon'
ordf16['yr']='2016'
ordf16['State']='Oregon'
ordf15['yr']='2015'
ordf15['State']='Oregon'

wadf17['yr']='2017'
wadf17['State']='Washington'
wadf16['yr']='2016'
wadf16['State']='Washington'
wadf15['yr']='2015'
wadf15['State']='Washington'

In [6]:
#concatenate table for california weather into one table
ca_results = pd.concat([cadf17, cadf16, cadf15], axis=0, sort=False).reset_index(drop=True)
ca_results.reset_index(level=0, inplace=True)
ca_results = ca_results.rename(index=str, columns={"level_0": "id","index": "measurement"})
ca_results

,id,measurement,Avg,yr,State
0,0,Avg Temperature,69.333333,2017,California
1,1,Max Temperature,78.916667,2017,California
2,2,Min Temperature,62.416667,2017,California
3,3,Avg Temperature,67.333333,2016,California
4,4,Max Temperature,76.416667,2016,California
5,5,Min Temperature,59.583333,2016,California
6,6,Avg Temperature,68.416667,2015,California
7,7,Max Temperature,78.000000,2015,California
8,8,Min Temperature,60.500000,2015,California


In [7]:
#concatenate table for Oregon weather into one table
or_results = pd.concat([ordf17, ordf16, ordf15], axis=0, sort=False).reset_index(drop=True)
or_results.reset_index(level=0, inplace=True)
or_results = or_results.rename(index=str, columns={"level_0": "id","index": "measurement"})
or_results

,id,measurement,Avg,yr,State
0,0,Avg Temperature,54.666667,2017,Oregon
1,1,Max Temperature,65.416667,2017,Oregon
2,2,Min Temperature,45.083333,2017,Oregon
3,3,Avg Temperature,55.083333,2016,Oregon
4,4,Max Temperature,64.916667,2016,Oregon
5,5,Min Temperature,46.166667,2016,Oregon
6,6,Avg Temperature,55.833333,2015,Oregon
7,7,Max Temperature,66.666667,2015,Oregon
8,8,Min Temperature,45.916667,2015,Oregon


In [8]:
#concatenate table for Washington weather into one table
wa_results = pd.concat([wadf17, wadf16, wadf15], axis=0, sort=False).reset_index(drop=True)
wa_results.reset_index(level=0, inplace=True)
wa_results = wa_results.rename(index=str, columns={"level_0": "id","index": "measurement"})
wa_results

,id,measurement,Avg,yr,State
0,0,Avg Temperature,53.333333,2017,Washington
1,1,Max Temperature,62.583333,2017,Washington
2,2,Min Temperature,45.916667,2017,Washington
3,3,Avg Temperature,55.333333,2016,Washington
4,4,Max Temperature,64.250000,2016,Washington
5,5,Min Temperature,48.083333,2016,Washington
6,6,Avg Temperature,57.666667,2015,Washington
7,7,Max Temperature,67.333333,2015,Washington
8,8,Min Temperature,48.916667,2015,Washington


# Load to Relational database (SQL)

## Connect to local database

In [11]:
#connect to local database
rds_connection_string = "<inser user name>:<insert password>@127.0.0.1/weather_db"
engine = create_engine(f'mysql://{rds_connection_string}')

AttributeError: module 'config' has no attribute '_instantiate_plugins'

## Using MYSQL Workbench: Create database and empty table

CREATE DATABASE weather_db;

USE weather_db;


CREATE TABLE ca_sqlresults (
 id INT PRIMARY KEY,
  measurement VARCHAR(30) NOT NULL,
  Avg INT,
  yr INT,
  State VARCHAR(30) NOT NULL
);

CREATE TABLE or_rsqlesults (
 id INT PRIMARY KEY,
  measurement VARCHAR(30) NOT NULL,
  Avg INT,
  yr INT,
  State VARCHAR(30) NOT NULL
);

CREATE TABLE wa_sqlresults (
 id INT PRIMARY KEY,
  measurement VARCHAR(30) NOT NULL,
  Avg INT,
  yr INT,
  State VARCHAR(30) NOT NULL
);

## Check for tables

In [ ]:
engine.table_names()

## Use pandas to load csv converted DataFrame into database

In [ ]:
ca_results.to_sql(name='ca_sqlresults', con=engine, if_exists='append', index=False)
or_results.to_sql(name='or_sqlresults', con=engine, if_exists='append', index=False)
wa_results.to_sql(name='wa_sqlresults', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the tables

In [ ]:
pd.read_sql_query('SELECT * FROM ca_sqlresults', con=engine)

In [ ]:
pd.read_sql_query('SELECT * FROM or_sqlresults', con=engine)

In [ ]:
pd.read_sql_query('SELECT * FROM wa_sqlresults', con=engine)